In [1]:
!pip install tensorflow-gpu==2.0.0-rc1

     |████████████████████████████████| 380.5MB 75kB/s 
     |████████████████████████████████| 501kB 50.5MB/s 
     |████████████████████████████████| 4.3MB 51.6MB/s 


In [2]:
import tensorflow as tf
tf.__version__

'2.0.0-rc1'

In [3]:
matrix1 = tf.constant([[3., 3.]])

matrix2 = tf.constant([[2.],[2.]])

matrix1, matrix2

(<tf.Tensor: id=0, shape=(1, 2), dtype=float32, numpy=array([[3., 3.]], dtype=float32)>,
 <tf.Tensor: id=1, shape=(2, 1), dtype=float32, numpy=
 array([[2.],
        [2.]], dtype=float32)>)

In [4]:
product = tf.matmul(matrix1, matrix2)
product

<tf.Tensor: id=2, shape=(1, 1), dtype=float32, numpy=array([[12.]], dtype=float32)>

In [5]:
matrix1 @ matrix2

<tf.Tensor: id=3, shape=(1, 1), dtype=float32, numpy=array([[12.]], dtype=float32)>

In [6]:
product + 3

<tf.Tensor: id=5, shape=(1, 1), dtype=float32, numpy=array([[15.]], dtype=float32)>

In [7]:
w = tf.Variable(shape=(1, 2), initial_value=[[2., 1.]])
w

<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[2., 1.]], dtype=float32)>

In [8]:
y = w @ [[1], [2]]
y

<tf.Tensor: id=18, shape=(1, 1), dtype=float32, numpy=array([[4.]], dtype=float32)>

In [9]:
with tf.GradientTape() as tape:
  y = w@[[1], [2]]
  loss = (y - 3)**2
gradients = tape.gradient(loss, [w])
gradients

[<tf.Tensor: id=34, shape=(1, 2), dtype=float32, numpy=array([[2., 4.]], dtype=float32)>]

## MNIST Again

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = (x_train-127.5)/127.5
x_test = (x_test-127.5)/127.5


In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(10000).batch(32)


In [0]:
train_ds

In [0]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.shuffle(10000).batch(32)



In [0]:
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.models import Model

class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')    
    self.d2 = Dense(10, activation='softmax')

  def call(self, x): 
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()


In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [0]:
#@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)


In [0]:
#@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()
